<a href="https://colab.research.google.com/github/abreham-gebremedin/Machine-Learning/blob/main/Fake_News_Classifier_Using_Bidirectional_LSTM_and_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Abreham Gebremedin
 (https://abreham-gebremedin.herokuapp.com/)

# Fake News Classifier Using Bidirectional LSTM
Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
!pip install opendatasets
 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import opendatasets as od
import pandas as pd
  
od.download(
    "https://www.kaggle.com/c/fake-news/data?select=train.csv")

Skipping, found downloaded files in "./fake-news" (use force=True to force download)


In [3]:


df=pd.read_csv('/content/fake-news/train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


###Drop Nan Values


In [5]:
df=df.dropna()

## Get the Independent Features


In [6]:



X=df.drop('label',axis=1)



## Get the Dependent features


In [7]:


y=df['label']



In [8]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [9]:
X.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.8.2'

In [13]:


from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout



### Vocabulary size


In [14]:


voc_size=5000




# Onehot Representation


In [15]:
messages=X.copy()

In [16]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [17]:
messages.reset_index(inplace=True)

In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Dataset Preprocessing


In [20]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)



In [21]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [22]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3397, 2256, 226, 4490, 3649, 4452, 3747, 4182, 3778, 1248],
 [1973, 4095, 1397, 643, 3703, 303, 188],
 [1822, 529, 311, 4885],
 [1086, 3446, 4754, 3002, 819, 4133],
 [719, 3703, 3357, 2495, 941, 3881, 3703, 17, 528, 4200],
 [1122,
  2920,
  3661,
  4844,
  1338,
  3570,
  1737,
  4246,
  2768,
  1398,
  2686,
  150,
  709,
  1141,
  188],
 [3493, 3267, 2944, 1172, 707, 1387, 4807, 3788, 445, 2774, 3882],
 [3094, 2802, 767, 4592, 735, 1360, 3570, 3177, 445, 2774, 3882],
 [145, 4615, 2536, 1130, 3108, 4456, 1635, 781, 3570, 1775],
 [2839, 1738, 2184, 1558, 1215, 3289, 3949, 4610],
 [738, 1031, 4416, 4808, 4281, 2398, 2717, 2709, 3752, 3615, 698],
 [3002, 3067, 3649, 4456, 3570, 735],
 [1204, 1588, 1117, 3712, 3281, 4790, 34, 1195, 658],
 [2522, 4594, 3206, 2970, 1898, 4318, 1673, 445, 2774, 3882],
 [4032, 658, 1406, 2238, 585, 445, 2774, 3882],
 [2854, 1489, 1111, 1763, 3927, 3044, 2183, 4990, 4612, 1597],
 [4855, 3937, 4095],
 [806, 609, 3132, 2116, 3570, 2980, 2835, 188],
 [3239, 396


# Embedding Representation


In [23]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4182 3778 1248]
 [   0    0    0 ... 3703  303  188]
 [   0    0    0 ...  529  311 4885]
 ...
 [   0    0    0 ...  445 2774 3882]
 [   0    0    0 ... 3702 3213 1648]
 [   0    0    0 ...  643 4974 4832]]


In [24]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3397,
       2256,  226, 4490, 3649, 4452, 3747, 4182, 3778, 1248], dtype=int32)

## Creating LSTM model


In [25]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


## Creating  Bidirectional LSTM model


In [26]:


embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
len(embedded_docs),y.shape

(18285, (18285,))

In [28]:


import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)



In [29]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

##Train -Test split

In [30]:


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)




# Model Training


### LSTM *Model* Training


In [31]:


model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)



Epoch 1/10
192/192 [==============================] - 9s 36ms/step - loss: 0.3314 - accuracy: 0.8417 - val_loss: 0.1933 - val_accuracy: 0.9185
Epoch 2/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1420 - accuracy: 0.9442 - val_loss: 0.2065 - val_accuracy: 0.9213
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1024 - accuracy: 0.9631 - val_loss: 0.2160 - val_accuracy: 0.9201
Epoch 4/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0739 - accuracy: 0.9738 - val_loss: 0.2492 - val_accuracy: 0.9152
Epoch 5/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0517 - accuracy: 0.9828 - val_loss: 0.2699 - val_accuracy: 0.9122
Epoch 6/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0371 - accuracy: 0.9881 - val_loss: 0.3499 - val_accuracy: 0.9130
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0201 - accuracy: 0.9937 - val_loss: 0.3932 - val_accuracy: 0.9130

### Bidirectional LSTM model Training

In [32]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 15s 58ms/step - loss: 0.3024 - accuracy: 0.8547 - val_loss: 0.2093 - val_accuracy: 0.9138
Epoch 2/10
192/192 [==============================] - 9s 48ms/step - loss: 0.1426 - accuracy: 0.9453 - val_loss: 0.1988 - val_accuracy: 0.9165
Epoch 3/10
192/192 [==============================] - 9s 47ms/step - loss: 0.1024 - accuracy: 0.9632 - val_loss: 0.2367 - val_accuracy: 0.9090
Epoch 4/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0769 - accuracy: 0.9731 - val_loss: 0.2442 - val_accuracy: 0.9099
Epoch 5/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0584 - accuracy: 0.9802 - val_loss: 0.3396 - val_accuracy: 0.9097
Epoch 6/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0419 - accuracy: 0.9858 - val_loss: 0.4374 - val_accuracy: 0.9094
Epoch 7/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0263 - accuracy: 0.9918 - val_loss: 0.4660 - val_accuracy: 0.903

In [33]:
from sklearn.metrics import confusion_matrix


## Performance Metrics And Accuracy for LSTM model


In [34]:
results = model.evaluate(X_test,y_test)
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 7ms/step - loss: 0.5576 - accuracy: 0.9132


## Performance Metrics And Accuracy for *Bidirectional* LSTM model


In [36]:
results = model1.evaluate(X_test,y_test)
y_pred=model1.predict(X_test)

189/189 [==============================] - 4s 19ms/step - loss: 0.5318 - accuracy: 0.8973
